In [145]:
import pandas as pd
from pathlib import Path
from os import listdir

In [146]:
repo_dir = Path("/home/luk/forks/RAPid-Learn/data")
experiment_id = "2021-09-06_13:37:28-baseline-PPO-50000episodes-rewardshapingon-68074bfd5da343678bbacc75a1dbdb70"
filename = repo_dir / experiment_id / "firecraftingtablehard-monitor.csv"

In [155]:
TARGET_DIR = repo_dir / experiment_id / "parsed_results"
TARGET_DIR.mkdir(parents=True, exist_ok=True)

In [148]:
csvs = [filename for filename in listdir(repo_dir / experiment_id) if filename.endswith(".csv")]; csvs

['firecraftingtablehard-monitor.csv',
 'rubbertree-monitor.csv',
 '2021-09-06_13:37:31-monitor.csv']

In [157]:
def parse_results(df):
    df["success"] = df["success"].astype(int)    
    iter_df = df.copy(deep=True)
    previous_row = None
    in_test_block = False
    n_test_blocks = 0
    len_test_block = 20
    current_eval_episode = -1

    df["in_test_block"] = False
    df["eval_episode"] = -1

    for i, row in iter_df.iterrows():
        if previous_row is None:
            previous_row = row
            continue
        if 'train' in previous_row["mode"] and 'train' not in row["mode"]:
            in_test_block = True
            current_eval_episode = i - 1 - (n_test_blocks * len_test_block)
            n_test_blocks += 1
        if 'train' not in previous_row["mode"] and 'train' in row["mode"]:
            in_test_block = False
            current_eval_episode = -1
        df["eval_episode"].iloc[i] = current_eval_episode
        df["in_test_block"].iloc[i] = in_test_block
        previous_row = row

    df = df.loc[df['in_test_block']]

    output = df.groupby("eval_episode").agg({'r':['mean','std'],'success':['mean','std']})
    output.columns = [' '.join(col).strip() for col in output.columns.values]
    return output

In [156]:
for csv in csvs:
    print(f"Now parsing results for {csv}!")
    df = pd.read_csv(repo_dir / experiment_id / csv, skiprows=[0])
    output = parse_results(df)
    output.to_csv(TARGET_DIR / f"parsed-{csv}")

Now parsing results for firecraftingtablehard-monitor.csv!
entering test block
exiting test block
entering test block
exiting test block
entering test block
Now parsing results for rubbertree-monitor.csv!
Now parsing results for 2021-09-06_13:37:31-monitor.csv!
